In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def load_json_data(file_path):
    data_file = open(file_path)
    data = []
    for line in data_file:
        data.append(json.loads(line))
    df = pd.DataFrame(data)
    data_file.close()
    return df

In [3]:
def print_df_info(df):
    print("Columns:", df.columns)
    print("Total Rows:", len(df))
    for column in df.columns:
        print(f"{column} Unique Rows:", df[column].nunique())

def print_df_null_info(df):
    null_vals = ['na', 'null', 'none', 'nan', '', ' ']
    for column in df.columns:
        print(f"{column} Number of Nulls:", df[column].isnull().sum())
        if pd.api.types.infer_dtype(df[column]) == 'string':
            print(f"{column} Number of string Nulls:", df[column].str.lower().isin(null_vals).sum())

In [4]:
# Loading all data took about ~25 GB of memory
dataset_folder = "yelp_dataset"
checkin_path = f"{dataset_folder}/yelp_academic_dataset_checkin.json"
business_path = f"{dataset_folder}/yelp_academic_dataset_business.json"
review_path = f"{dataset_folder}/yelp_academic_dataset_review.json"
tips_path = f"{dataset_folder}/yelp_academic_dataset_tip.json"
user_path = f"{dataset_folder}/yelp_academic_dataset_user.json" 

In [ ]:
df_checkin = load_json_data(checkin_path)
# df_business = load_json_data(business_path)
# df_review = load_json_data(review_path)
# df_tips = load_json_data(tips_path)
# df_user = load_json_data(user_path)

In [73]:
# business_id: string, 22 character business id, maps to business in business.json
# date: string which is a comma-separated list of timestamps for each checkin, each with format YYYY-MM-DD HH:MM:SS
print_df_info(df_checkin)

Columns: Index(['business_id', 'date'], dtype='object')
Total Rows: 138876
business_id Unique Rows: 138876
business_id Number of Nulls: 0
business_id Number of string Nulls: 0
date Unique Rows: 138875
date Number of Nulls: 0
date Number of string Nulls: 0


In [77]:
csv_folder = 'yelp_dataset_csv'

In [76]:
# This is a multivalued attribute as such I treated each date value as its own separate row
# For example business_id: id_1, date: date_1, date_2 becomes row 1: id_1, date_1; row 2: id_1, date_2;
df_checkin = load_json_data(checkin_path)
df_checkin['date'] = df_checkin.date.str.split(',')
df_checkin = df_checkin.explode('date')
df_checkin.date = df_checkin.date.str.strip()
df_checkin.to_csv(f"{csv_folder}/yelp_academic_dataset_checkin.csv", index=False)

In [79]:
print_df_info(df_checkin)

Columns: Index(['business_id', 'date'], dtype='object')
Total Rows: 18641292
business_id Unique Rows: 138876
business_id Number of Nulls: 0
business_id Number of string Nulls: 0
date Unique Rows: 17748948
date Number of Nulls: 0
date Number of string Nulls: 0


In [80]:
df_checkin

,business_id,date
0,--0r8K_AQ4FZfLsX3ZYRDA,2017-09-03 17:13:59
1,--0zrn43LEaB4jUWTQH_Bg,2010-10-08 22:21:20
1,--0zrn43LEaB4jUWTQH_Bg,2010-11-01 21:29:14
1,--0zrn43LEaB4jUWTQH_Bg,2010-12-23 22:55:45
1,--0zrn43LEaB4jUWTQH_Bg,2011-04-08 17:14:59
...,...,...
138874,zzwK-TJsCJX5wZrdtKemPg,2012-10-02 14:32:18
138874,zzwK-TJsCJX5wZrdtKemPg,2012-11-29 22:23:01
138874,zzwK-TJsCJX5wZrdtKemPg,2014-04-24 23:22:41
138875,zzzKmD9Mj6WtJwJUhA_1dg,2010-12-25 22:17:04


In [86]:
df_business = load_json_data(business_path)

In [95]:
# business_id: string, 22 character unique string business id
# name: string, the business's name
# address: string, the full address of the business
# city: string, the city
# state: string, 2 character state code, if applicable
# postal code: string, the postal code
# latitude: float, latitude
# longitude: float, longitude
# stars: float, star rating, rounded to half-stars
# review_count: integer, number of reviews
# is_open: integer, 0 or 1 for closed or open, respectively
# attributes: object, business attributes to values. note: some attribute values might be objects
# categories: an array of strings of business categories
# hours: an object of key day to value hours, hours are using a 24hr clock
# NOTE attributes and hours are dictionaries

print_df_info(df_business[df_business.columns.drop(['attributes','hours'])])

Columns: Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'categories'],
      dtype='object')
Total Rows: 160585
business_id Unique Rows: 160585
name Unique Rows: 125850
address Unique Rows: 123895
city Unique Rows: 836
state Unique Rows: 31
postal_code Unique Rows: 5779
latitude Unique Rows: 137397
longitude Unique Rows: 133643
stars Unique Rows: 9
review_count Unique Rows: 1281
is_open Unique Rows: 2
categories Unique Rows: 88115


In [125]:
# Hours is a multi-valued attribute so I treated it as a separate entity / csv file
df_business_hours = df_business[['business_id','hours']].copy()
df_business_hours = df_business_hours.join(pd.DataFrame(df_business_hours.hours.to_dict()).T)
df_business_hours = df_business_hours[df_business_hours.columns.drop('hours')]
df_business_hours = df_business_hours.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_business_hours

,business_id,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,6iYb2HFDywm3zjuRg0shjw,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0,11:0-23:0
1,tCbdrRPZA0oiIYSmHG3J0w,5:0-18:0,5:0-17:0,5:0-18:0,5:0-18:0,5:0-18:0,5:0-18:0,5:0-18:0
2,bvN78flM8NLprQ1a1y5dRg,NaN,NaN,NaN,11:0-18:0,11:0-18:0,11:0-18:0,11:0-18:0
3,oaepsyvc0J17qwi8cfrOWg,None,None,None,None,None,None,None
4,PE9uqAjdw0E4-8mjGl3wVA,16:0-19:0,16:0-19:0,16:0-19:0,16:0-19:0,16:0-19:0,9:0-11:0,NaN
...,...,...,...,...,...,...,...,...
160580,D2mHoIDXx9N8mS1pGoKV9Q,9:0-19:0,9:0-19:0,9:0-19:0,9:0-19:0,9:0-19:0,9:0-19:0,9:0-19:0
160581,bQX-kwVTyZgcdZGEPzce6Q,10:0-18:0,10:0-18:0,10:0-18:0,10:0-18:0,10:0-18:0,9:0-17:0,NaN
160582,wvFZ06nmPmQ2-IVoPqVYLA,0:0-0:0,NaN,12:0-17:0,19:0-21:0,19:0-21:0,15:30-21:0,16:0-18:0
160583,GB75wPibj3IjNauaoCxyGA,None,None,None,None,None,None,None


In [126]:
df_business_hours.to_csv(f'{csv_folder}/yelp_academic_dataset_business_hours.csv', index=False)

In [117]:
df_business_hours.describe()

,business_id,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
count,160585,121672,126944,129469,130753,130177,113021,83780
unique,160585,1408,1509,1515,1563,1613,1509,1385
top,COx7XZCMnmj9wNzNIILD3A,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0,0:0-0:0
freq,1,28859,5905,5906,5888,5922,5942,5897


In [123]:
print_df_null_info(df_business_hours)

business_id Number of Nulls: 0
business_id Number of string Nulls: 0
Monday Number of Nulls: 38913
Monday Number of string Nulls: 0
Tuesday Number of Nulls: 33641
Tuesday Number of string Nulls: 0
Wednesday Number of Nulls: 31116
Wednesday Number of string Nulls: 0
Thursday Number of Nulls: 29832
Thursday Number of string Nulls: 0
Friday Number of Nulls: 30408
Friday Number of string Nulls: 0
Saturday Number of Nulls: 47564
Saturday Number of string Nulls: 0
Sunday Number of Nulls: 76805
Sunday Number of string Nulls: 0


In [120]:
# Checking to make sure null values were captured appropriately
df_business_hours[df_business_hours.Monday.isnull()]

,business_id,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
2,bvN78flM8NLprQ1a1y5dRg,NaN,NaN,NaN,11:0-18:0,11:0-18:0,11:0-18:0,11:0-18:0
3,oaepsyvc0J17qwi8cfrOWg,None,None,None,None,None,None,None
7,jFYIsSb7r1QeESVUnXPHBw,NaN,NaN,NaN,NaN,NaN,8:0-14:0,8:0-14:0
13,ufCxltuh56FF4-ZFZ6cVhg,NaN,11:0-18:0,11:0-18:0,11:0-18:0,11:0-18:0,11:0-18:0,NaN
14,i_t_30RYVUDdZzFIcw80NQ,NaN,9:0-17:30,9:0-17:30,9:0-17:30,9:0-17:30,9:0-17:30,NaN
...,...,...,...,...,...,...,...,...
160558,HRoRvwxUwgNA2L0qiMRVMw,None,None,None,None,None,None,None
160561,jYgqSazE0gUyI7qq086Dzw,NaN,16:0-21:0,16:0-21:0,16:0-21:0,11:30-21:0,11:30-21:0,11:30-21:0
160570,det-KjTfYoG2JfCZG3v04w,None,None,None,None,None,None,None
160583,GB75wPibj3IjNauaoCxyGA,None,None,None,None,None,None,None


In [87]:
df_business = df_business[df_business.columns.drop('hours')]

In [7]:
df_business_categories = df_business[['business_id','categories']].copy()
df_business_categories.categories = df_business_categories.categories.str.split(',')
df_business_categories

,business_id,categories
0,6iYb2HFDywm3zjuRg0shjw,"[Gastropubs, Food, Beer Gardens, Restaurant..."
1,tCbdrRPZA0oiIYSmHG3J0w,"[Salad, Soup, Sandwiches, Delis, Restauran..."
2,bvN78flM8NLprQ1a1y5dRg,"[Antiques, Fashion, Used, Vintage & Consign..."
3,oaepsyvc0J17qwi8cfrOWg,"[Beauty & Spas, Hair Salons]"
4,PE9uqAjdw0E4-8mjGl3wVA,"[Gyms, Active Life, Interval Training Gyms, ..."
...,...,...
160580,D2mHoIDXx9N8mS1pGoKV9Q,"[Real Estate, Real Estate Services, Home Ser..."
160581,bQX-kwVTyZgcdZGEPzce6Q,"[Health Markets, Food, Specialty Food, Groc..."
160582,wvFZ06nmPmQ2-IVoPqVYLA,"[Arts & Entertainment, Paint & Sip, Art Clas..."
160583,GB75wPibj3IjNauaoCxyGA,"[Cuban, Sandwiches, Restaurants, Cafes]"


In [67]:
def check_list(row, string):
    check = False
    if row:
        for elem in row:
            if string in elem:
                check = True
    return check
df_check = df_business_categories.categories.apply(lambda row: check_list(row, '& Pro'))

In [69]:
df_test = df_business_categories[df_check]

In [48]:
categories_list = (df_business_categories.categories.values)

def flatten_list(two_dim_list):
    flat_list = []
    for row in two_dim_list:
        if row:
            try:
                for element in row:
                    flat_list.append(element.strip())
            except:
                print(row)
    return flat_list 

categories = flatten_list(categories_list)
print(len(categories))

708968


In [9]:
category_types = (list(set(categories)))
category_types = pd.DataFrame({'category_types':category_types})
category_types = category_types.sort_values('category_types')
category_types

,category_types
21,& Probates
1081,3D Printing
159,ATV Rentals/Tours
248,Acai Bowls
681,Accessories
...,...
742,Yakiniku
534,Yelp Events
963,Yoga
125,Ziplining


In [73]:
def strip_row(row):
    if row:
        return [elem.strip() for elem in row]

df_business_categories = df_business[['business_id','categories']].copy()
df_business_categories.categories = df_business_categories.categories.str.split(',')
df_business_categories.categories = df_business_categories.categories.apply(lambda row: strip_row(row))

empty_indices = df_business_categories[df_business_categories.categories.isnull()].index.values
for index in empty_indices:
    df_business_categories.iloc[index].categories = []

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(sparse_output=True)

df_business_categories = df_business_categories.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(df_business_categories.pop('categories')),
                index=df_business_categories.index,
                columns=mlb.classes_))


In [78]:
df_business_categories.to_csv(f'{csv_folder}/yelp_academic_dataset_business_categories.csv', index=False)

In [88]:
df_business = df_business[df_business.columns.drop('categories')]

In [89]:
df_business_attributes = df_business[['business_id', 'attributes']].copy()

In [90]:
# df_business = load_json_data(business_path)
df_business_attributes = df_business_attributes.join(pd.DataFrame(df_business_attributes.attributes.to_dict()).T)
df_business_attributes = df_business_attributes[df_business_attributes.columns.drop('attributes')]
df_business_attributes = df_business_attributes.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_business_attributes

,business_id,RestaurantsTableService,WiFi,BikeParking,BusinessParking,BusinessAcceptsCreditCards,RestaurantsReservations,WheelchairAccessible,Caters,OutdoorSeating,...,BYOB,CoatCheck,Smoking,DriveThru,BYOBCorkage,Corkage,RestaurantsCounterService,AgesAllowed,DietaryRestrictions,Open24Hours
0,6iYb2HFDywm3zjuRg0shjw,True,u'free',True,"{'garage': False, 'street': True, 'validated':...",True,False,True,True,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tCbdrRPZA0oiIYSmHG3J0w,NaN,u'free',False,"{'garage': True, 'street': False, 'validated':...",True,False,NaN,True,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bvN78flM8NLprQ1a1y5dRg,NaN,NaN,False,"{'garage': False, 'street': True, 'validated':...",True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,oaepsyvc0J17qwi8cfrOWg,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,PE9uqAjdw0E4-8mjGl3wVA,NaN,NaN,NaN,"{'garage': False, 'street': False, 'validated'...",True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160580,D2mHoIDXx9N8mS1pGoKV9Q,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
160581,bQX-kwVTyZgcdZGEPzce6Q,NaN,NaN,True,"{'garage': False, 'street': False, 'validated'...",True,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160582,wvFZ06nmPmQ2-IVoPqVYLA,NaN,u'no',NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160583,GB75wPibj3IjNauaoCxyGA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
df_business_attributes[0:3].T

,0,1,2
business_id,6iYb2HFDywm3zjuRg0shjw,tCbdrRPZA0oiIYSmHG3J0w,bvN78flM8NLprQ1a1y5dRg
RestaurantsTableService,True,NaN,NaN
WiFi,u'free',u'free',NaN
BikeParking,True,False,False
BusinessParking,"{'garage': False, 'street': True, 'validated':...","{'garage': True, 'street': False, 'validated':...","{'garage': False, 'street': True, 'validated':..."
BusinessAcceptsCreditCards,True,True,True
RestaurantsReservations,False,False,NaN
WheelchairAccessible,True,NaN,NaN
Caters,True,True,NaN
OutdoorSeating,True,False,NaN


In [99]:
df_business_attributes_business_parking = df_business_attributes[['business_id','BusinessParking']]
df_business_attributes_business_parking.BusinessParking[0:5].T

0    {'garage': False, 'street': True, 'validated':...
1    {'garage': True, 'street': False, 'validated':...
2    {'garage': False, 'street': True, 'validated':...
3                                                  NaN
4    {'garage': False, 'street': False, 'validated'...
Name: BusinessParking, dtype: object